In [ ]:
from numpy.random import randn, rand, standard_t, normal, uniform
from numpy import sqrt, arange, mean, std
from numpy.linalg import inv
from scipy.optimize import leastsq                       # Minimize the sum of squares of a set of equations. leastsq internamente usa um método gradiente de Levenburg-Marquardt para minimizar a função objetivo
from scipy.optimize import least_squares                 # least_squares finds a local minimum of the cost function F(x)
from scipy.optimize import curve_fit                     # Use non-linear least squares to fit a function, f, to data
import pandas as pd                                      # Carrega o pacote pandas e o chama de pd
import numpy as np                                       # Carrega o pacote pandas e o chama de pd
np.set_printoptions(precision=4)


### NLLS

Suponha que queiramos estimar o seguinte modelo não linear:
$$ y_i = \beta_0 + x_i^{\beta_1} + e_i$$

Para isso, vamos usar $n=1000$ observações construídas artificialmente com $\beta_0=0.5$ e $\beta_1=1.5$:

In [ ]:
n  = 1000;
e  = uniform(-1,1,n);                  # Simula n choques uniformes em [-1,1]
x  = normal(10,2,n);                   # simula uma variável normal com média 10, std=2
y  = .5  +  x**(1.5) + e               # constrói y

### Definindo a função objetivo:

In [ ]:
def NLLS_obj(beta, y, x):          # Cria um objeto "NLLS_obj" que contém a função objetivo do problema
    b_0 = beta[0]
    b_1 = beta[1]
    return y - b_0 -  (x**b_1)     # retorna o resultado da conta y - b0 - b1 * (x**b2)

Agora, precisamos definir um chute inicial. Ele precisa ser tal que y exista!

In [ ]:
beta0   = np.array([1.0,1.0])        # Chute inicial
Beta_nl = least_squares(NLLS_obj, beta0, args = (y, x))  # least_squares(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
Beta_nl = Beta_nl.x

Beta_nl

Agora, precisamos calcular a jacobinada para estimarmos a variância de $\beta$
Note, inicialmente, que se trata de um modelo não linear, exatamente identificado e homocedástico. Além disso, 

$$\theta=(\beta_0,\beta_1)^\top$$
$$ X_i = (1,x_i)^\top$$
\begin{equation*}
	g(v_i,\theta)=y_i-\beta_0 -x_i^{\beta_1}
\end{equation*}

$$\underset{(1\times K)}{\partial_\theta g(v_i;\theta)}=\begin{bmatrix}
-1 & - x_i^{\beta_1}\text{log}(x_i)\\ 
\end{bmatrix}$$

In [ ]:
g_11  = - np.ones((n,1))                     # Gradiente del_g1/del beta_0
g_12  = - (x**Beta_nl[1])*(np.log(x));       # Gradient  del_g1/del beta_1
g_12  = g_12[...,None]
dG    = np.concatenate((g_11,g_12),axis=1)


In [ ]:
e_hat = (y - Beta_nl[0] - x**Beta_nl[1])
sig_2 = e_hat.T@e_hat/(n-2)
sig_2

In [ ]:
Var_B_nl = sig_2 * inv(dG.T@dG)
print(Var_B_nl)
SE_nl    = sqrt(np.diag(Var_B_nl))
SE_nl

In [ ]:
def NLLS_obj2(x,b_0,b_1):        # Cria um objeto "NLLS_obj2" que contém a função objetivo do problema
    return b_0 + (x**b_1)        # retorna o resultado da conta b0 + b1 * (x**b2)

In [ ]:
## Organizando as variáveis para o print
np.set_printoptions(precision=4)
popt, pcov = curve_fit(NLLS_obj2, x, y)

print("Parameter estimation results:")
print("b_0 = "," %8.4f" % popt[0]," | b_1 = "," %8.4f" % popt[1])
print("------------------------------------------------------")
print("------------------------------------------------------")
print("Standard Deviations:")
print(sqrt(np.diag(pcov)))

### GMM Não linear



\begin{eqnarray*}
	g_1(v_i, \theta_0)&=& z_{i1}(y_i-f(x_i,\theta_0))= 0\\
	g_2(v_i, \theta_0)&=& z_{i2}(y_i-f(x_i,\theta_0))= 0\\
	\vdots&=& \vdots \\
	g_m(v_i, \theta_0)&=& z_{im}(y_i-f(x_i,\theta_0))= 0,
\end{eqnarray*}




## Modelo

Vamos simplificar o modelo para fins de ilustração. -- ou precisaríamos modelar os instrumentos, o que foge ao escopo da aula.

Defina o distúrbio aleatório por 

$$\epsilon_i=y_i-\beta_0 -x_i^{\beta_0}$$

Note que agora $b_1=b_0$ e, portando, temos um parâmetro e duas variáveis explicativas: a constante e $x$.

Com isso, podemos utilizar o GMM para recuperar $\beta_0$.


Seja 
$$\theta=\beta_0$$
com 
$$ X_i = x_i$$
e o conjunto de instrumentos
$$ Z_i = (1,x_i)^\top$$

Note-se, inicialmente, que se trata de um modelo não linear, sobre-identificado e homocedástico. Para ter uma solução, assume-se que $x_i>0$.  As condições de momento do problema são as seguintes:

\begin{equation*}
	g(v_i,\theta_0)=Z_i(y_i-\beta_0 -x_i^{\beta_0})=\begin{pmatrix}
		1(y_i-\beta_0 -x_i^{\beta_0})\\ 
		x_i(y_i-\beta_0 -x_i^{\beta_0})
	\end{pmatrix}
\end{equation*}

Com isso, o estimador de GMM de $\theta_0=\beta_0$ é dado pela minimização do seguinte problema:

$$ \underset{(K\times1)}{ \theta_{gmm}} = \underset{\theta_0}{\text{argmin }} \underset{(1\times m)}{\Bigg(\sum_{i=1}^n g(v_i;\theta_0)\Bigg)^\prime} \underset{(m\times m)}{W_n} \underset{(m\times 1)}{\Bigg(\sum_{i=1}^n g(v_i;\theta_0)\Bigg)} $$

ou seja, 


\begin{equation*}
	\hat{\theta}=\underset{\theta_0}{\text{arg min}} \begin{pmatrix}\frac{1}{n}\sum_{i=1}^n\begin{pmatrix}
		y_i-\beta_0 -x_i^{\beta_0}\\ 
		x_i(y_i-\beta_0 -x_i^{\beta_0})
	\end{pmatrix}\end{pmatrix}^\top W_n 	\begin{pmatrix}\frac{1}{n}\sum_{i=1}^n\begin{pmatrix}
	y_i-\beta_0 -x_i^{\beta_0}\\ 
	x_i(y_i-\beta_0 -x_i^{\beta_0})
\end{pmatrix} \end{pmatrix},
\end{equation*}
em que $W_n$ é uma matriz quadrada e positiva semi-definida.  

Pode-se mostrar que, assimptoticamente, vale que:

\begin{eqnarray*} \small
		&&\sqrt{n}(\hat{\theta}-\theta _{0})\overset{d}{\rightarrow }N(0,\left[
		E(\partial _{\theta }g(v_{i};\theta _{0}))^{\top }WE(\partial _{\theta
		}g(v_{i};\theta _{0}))\right]^{-1}E(\partial _{\theta }g(v_{i};\theta
		_{0}))^{\top }WE(g(v_{i};\theta _{0})g(v_{i};\theta _{0}))^{\top } \\
		&&{\quad \quad  \quad\quad \quad \quad  \quad \quad \quad \quad \quad \quad \quad \quad \quad }WE(\partial _{\theta
		}g(v_{i};\theta _{0}))\left[ E(\partial _{\theta }g(v_{i};\theta
		_{0}))^{\top }WE(\partial _{\theta }g(v_{i};\theta _{0}))\right] ^{-1})
\end{eqnarray*}

Sabe-se que o GMM torna-se eficiente quando a matriz de ponderação $W$ é dada pela 
inversa da variância dos momentos,
\begin{eqnarray*}
	W^{\ast } &=&\left[ E(g(v_{i};\theta _{0})g(v_{i};\theta _{0}))^{\prime }%
	\right] ^{-1}
\end{eqnarray*}
Neste caso, 
\begin{equation*}
	\sqrt{n}(\hat{\theta}-\theta _{0})\overset{d}{%
			\rightarrow }N\left( 0,\left[ E(\partial _{\theta }g(v_{i};\theta
		_{0}))^{\top }\left[ E(g(v_{i};\theta _{0})g(v_{i};\theta _{0}))^{\top }%
		\right] ^{-1}E(\partial _{\theta }g(v_{i};\theta _{0}))\right] ^{-1}\right) %
\end{equation*}


Com isso, usando 
$$W_n= E \Bigg[\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}^\top\Bigg]^{-1} $$ 
    
e a Jacobiana
    
$$G_0=\begin{pmatrix}
-1-x_{i}^{\beta_0}\text{ln}(x_{i})\\
-x_i-x_{i}{x_{i}^\top}^{\beta_0}\text{ln}(x_{i})
\end{pmatrix}$$ 
a variância assintótica de $\hat{\theta}$ será:


$$\text{Avar}(\hat{\theta})=\left[E\begin{pmatrix}
	-1-x_{i}^{\beta_0}\text{ln}(x_{i})\\
	-x_{i}-x_{i}{x_{i}^top}^{\beta_0}\text{ln}(x_{i})
\end{pmatrix}^\top E \Bigg[\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}^\top\Bigg]^{-1} 
E\begin{pmatrix}
	-1-x_{i}^{\beta_0}\text{ln}(x_{i})\\
	-x_{i}-x_{i}{x_{i}^\top}^{\beta_0}\text{ln}(x_{i})
\end{pmatrix}\right]^{-1}$$


Assim, nosso problema é encontrar as contra partidas amostrais $W_{n}^{\ast}$, $G_{0,n}$ tal que o estimador da variância assimptótica do estimador GMM eficiênte seja:

\begin{equation*}
	G_{0,n}^\top E \Bigg[\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}^\top\Bigg]^{-1}  G_{0,n}
\end{equation*}

Ou

\begin{equation*}
	\text{Avar}(\hat{\theta})=\left[ \frac{1}{n}\sum_{i=1}^n\begin{pmatrix}
		-1-x_{i}^{\beta_0}\text{ln}(x_{i})\\
		-x_{i}-x_{i}{x_{i}^\top}^{\beta_0}\text{ln}(x_{i})
	\end{pmatrix}^\top \frac{1}{n}\sum_{i=1}^n \Bigg[\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}\begin{pmatrix}
    1(y_i-\beta_0 -x_i^{\beta_0})\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}^\top\Bigg]^{-1} 
	\frac{1}{n}\sum_{i=1}^n\begin{pmatrix}
		-1-x_{i}^{\beta_0}\text{ln}(x_{i})\\
		-x_{i}-x_{i}{x_{i}^\top}^{\beta_0}\text{ln}(x_{i})
	\end{pmatrix}\right]^{-1}
\end{equation*}

Retomando, vamos construir os dados com o novo conjunto de parâmetros:

In [ ]:
m  = 2                                  # Número de condições de momento
n  = 1000;                              # Número de observações
e  = uniform(-1,1,(n,1));               # Simula n choques uniformes em [-1,1]
x  = normal(10,2,(n,1));                # simula uma variável normal com média 10, std=2
y  = 1.5  +  x**(1.5) + e               # constrói y

## Definindo a função objetivo:

In [ ]:
def GMM_obj(b0,y,x,W):                  # Cria um objeto "GMM_obj" que contém a função objetivo do problema
    g_1 = mean(y - b0 - x**b0);
    g_2 = mean(x*(y - b0 - x**b0));
    g   = np.array([g_1,g_2]);
    g   = np.reshape(g,(2,1));
    J   = g.T @ W @ g
    return  J[0]                         # retorna o valor da função objetivo J

In [ ]:
W  = np.eye(m)                           # Chute inicial: W= I_m

In [ ]:
def GMM_Var(b0,y,x): 
    G_1  = mean(-1 - x**b0*np.log(x));   # Derivada da condição de momento 1 com relação à b_0
    G_2  = mean(-x - x*x**b0*np.log(x)); # Derivada da condição de momento 2 com relação à b_0
    G    = np.array([G_1,G_2]);          # Constrói G_0
    G    = np.reshape(G,(2,1));          # Ajusta o objeto 1D para 2x1
    ehat = y - b0 - x**b0;               # Computa os resíduos da regressão       
    g_1  = (1*(ehat));                   # Condição de momento 1
    g_2  = (x*(ehat));                   # Condição de momento 2
    g    = np.concatenate((g_1,g_2),axis=1);       # Constrói g para n indivíduos. Portanto, mxn
    igg  = 1/n*g.T@g
    W    = inv(igg)
    AVAR = inv(G.T @ W @ G)
    return AVAR

## Computando $\beta$

In [ ]:
b_ini    = 0;
Beta_GMM = least_squares(GMM_obj,b_ini, args = (y,x,W),verbose=1)  # least_squares(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
Beta_GMM.x

In [ ]:
AVAR_GMM = GMM_Var(Beta_GMM.x,y,x)
SE_GMM   = sqrt(AVAR_GMM)
SE_GMM 

Agora, podemos construir W e obter uma estimação mais precisa de $\beta_0$ utilizando o 2SGMM:

In [ ]:
# Salvando os resíduos:
beta_GMM = Beta_GMM.x
ehat     = y - beta_GMM - x**beta_GMM; # resíduo da regressão

In [ ]:
# Forma 1: contra-partida das condições de momento
z    = np.concatenate((np.ones((n,1)),x),axis=1)
momt = z*ehat                     # Multiplicação elemento-por-elemento

In [ ]:
# Forma 2: Montando a matriz g item por item
g_1  = (1*(ehat));                # Condição de momento 1
g_2  = (x*(ehat));                # Condição de momento 2
g    = np.concatenate((g_1,g_2),axis=1);       # Constrói g para n indivíduos. Portanto, mxn


In [ ]:
g


In [ ]:
print(momt)

In [ ]:
igg  = 1/n*g.T@g
W    = inv(igg)
W

In [ ]:
## ou W = inv(1/n*momt.T@momt)
W = inv(1/n*(momt.T@momt))
W

In [ ]:
b_ini      = 0;
Beta_2SGMM = least_squares(GMM_obj,b_ini, args = (y,x,W),verbose=1)  # leastsq(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
Beta_2SGMM.x

In [ ]:
AVAR_2SGMM = GMM_Var(Beta_GMM.x,y,x)
SE_2SGMM   = sqrt(AVAR_2SGMM)
SE_2SGMM

## GMM-Interativo

Note que podemos continuar obtendo novas estimativas de $\beta_0$, pois agora poderemos obter uma nota matriz $W=W(\beta_{0,2SGMM})$.

De fato, podemos obter uma sequência de novas estimativas até que $W$ convirja para um valor fixo.

Há muitas formas para se implementar este estimador. Aqui, usaremos uma não tão elegante, mas que tem um carater pedagógico.


In [ ]:
Beta_GMM

In [ ]:
import copy
eps  = 1e-10                        # Define um valor suficientemente pequeno
dist = 10                           # Define um valor positivo suficientemente grande
Iter = 1;                           # Contador de iterações  
Beta_iGMM  = copy.deepcopy(Beta_GMM)# Copia o objeto Beta_GMM
Beta_iGMM.x=1;                      # Chute inicial 

In [ ]:
while dist > eps:
    if (Iter  == 1):
             W = np.eye(m)                    
    W_ini     = W                    # na iteração i, define W_inicial como o W que estiver na memória
    b_ini     = Beta_iGMM.x;          # na iteração i, define Beta_0_inicial como o Beta que estiver na memória
    Beta_iGMM = least_squares(GMM_obj,b_ini, args = (y,x,W),verbose=1)  # leastsq(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
    b0        = Beta_iGMM.x          # Salva o vetor estimado na iteração i como b0   
    ehat      = y - b0 - x**b0;      # Computa os resíduos da regressão       
    g_1       = (1*(ehat));          # Condição de momento 1
    g_2       = (x*(ehat));          # Condição de momento 2
    g         = np.array([g_1,g_2]); # Constrói g
    g         = np.reshape(g,(2,n)); # Ajusta o objeto 1D para 2x1 
    igg       = 1/n*g@g.T
    W         = inv(igg)
    diff      = abs(np.reshape(W, -1)-np.reshape(W_ini, -1)) # Computa o valor absoluto da diferença entre W's
    dist      = max(diff)
    Iter      = Iter+1

print(Iter)
print(dist)
print(b0-b_ini)
print(b0)

In [ ]:
dist = 10                           # Define um valor positivo suficientemente grande
Iter = 1;                           # Contador de iterações  
Beta_iGMM  = copy.deepcopy(Beta_GMM)# Copia o objeto Beta_GMM
Beta_iGMM.x=1;      

In [ ]:
while dist > eps:
    if (Iter  == 1):
             W = np.eye(m)        
    W_ini     = W                    # na iteração i, define W_inicial como o W que estiver na memória
    b_ini     = Beta_iGMM.x;         # na iteração i, define Beta_0_inicial como o Beta que estiver na memória
    Beta_iGMM = least_squares(GMM_obj,b_ini, args = (y,x,W),verbose=1)  # leastsq(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
    b0        = Beta_iGMM.x           # Salva o vetor estimado na iteração i como b0   
    ehat      = y - b0 - x**b0;      # Computa os resíduos da regressão       
    g_1       = (1*(ehat));          # Condição de momento 1
    g_2       = (x*(ehat));          # Condição de momento 2
    g         = np.array([g_1,g_2]); # Constrói g
    g         = np.reshape(g,(2,n)); # Ajusta o objeto 1D para 2x1 
    igg       = 1/n*g@g.T
    W         = inv(igg)
    diff      = abs(b0-b_ini) # Computa o valor absoluto da diferença entre Betas
    dist      = max(diff)
    Iter      = Iter+1

print(Iter)
print(dist)
print(b0-b_ini)
print(b0)

## CUGMM - Continuously updating GMM

Aqui, a ideia é estimar $\beta_0$ e $W(\beta_0)$ ao mesmo tempo. Isso requer resolver o seguinte problema não-linear:

\begin{equation*}
	\theta=\underset{\theta_0}{\text{arg min}}\left\{\frac{1}{n}\sum_{i=1}^n \begin{pmatrix}
		y_i-\beta_0 -x_i^{\beta_0}\\ 
		x_i(y_i-\beta_0 -x_i^{\beta_0})
	\end{pmatrix}^\top \frac{1}{n}\sum_{i=1}^n \Bigg[\begin{pmatrix}
    y_i-\beta_0 -x_i^{\beta_0}\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}\begin{pmatrix}
    y_i-\beta_0 -x_i^{\beta_0}\\ 
    x_i(y_i-\beta_0 -x_i^{\beta_0})
    \end{pmatrix}^\top\Bigg]^{-1}\frac{1}{n}\sum_{i=1}^n  	\begin{pmatrix}
	y_i-\beta_0 -x_i^{\beta_0}\\ 
	x_i(y_i-\beta_0 -x_i^{\beta_0})
\end{pmatrix} \right\},
\end{equation*}


Assim, definimos a nota função objetivo:

In [ ]:
def CUGMM_obj(b0,y,x):          # Cria um objeto "GMM_obj" que contém a função objetivo do problema
    n    = len(x)
    g_1  = (y - b0 - x**b0);
    g_2  = (x*(y - b0 - x**b0));
    g    = np.array([g_1,g_2]);
    g    = np.reshape(g,(n,2));
    momt = np.array([mean(g_1), mean(g_2)])
    momt = np.reshape(momt,(2,1));
    W    = inv(1/n*g.T@g)
    J    = momt.T @ W @ momt
    return  J[0]  

In [ ]:
b_ini      = Beta_iGMM.x;
Beta_CUGMM = least_squares(CUGMM_obj,b_ini, args = (y,x),verbose=1)  # leastsq(fun, x0, param) Minimiza a função fun com chute inicial x0 e parâmetros param
Beta_CUGMM.x

In [ ]:
AVAR_CUGMM = GMM_Var(Beta_CUGMM.x,y,x)
SE_CUGMM   = sqrt(AVAR_CUGMM)
SE_CUGMM

## Teste de sobreidentificação

Seriam válidos os momentos que assumimos para o modelo?
Se $m>K$, podemos testar se as condições de momento de fato possuem média 0.

Ou seja, podemos testar se 
$$ H_0 : E[g(v_i;\theta)] = 0$$
contra
$$ H_A : E[g(v_i;\theta)] \neq 0$$

O teste é intuitivo: se $\frac{1}{n}\sum_{i=1}^n g(v_i\theta)\xrightarrow{p}0$, então uma forma quadrática desta expressão terá, necessariamente, um valor pequeno e positivo. Disso, segue que a estatística de tal teste é chi-quadrado.


$$\text{$J$-stat}= n\bigg(\frac{1}{n}\sum_{i=1}^n g(v_i\theta)\bigg)' \hat{W} \bigg(\frac{1}{n}\sum_{i=1}^n g(v_i\theta)\bigg) \sim\chi^2_{m-K}$$


In [ ]:
from scipy.stats import chi2
z    = np.concatenate((np.ones((n,1)),x),axis=1)
ehat = y - Beta_CUGMM.x - x**Beta_CUGMM.x; # resíduo da regressão
g    = z*ehat
W    = inv(1/n*g.T@g)
momt = mean(g,axis=0)
momt = np.reshape(momt,(m,1)) 

In [ ]:
J_stat  = n*((1/n*momt.T) @ W @ (1/n* momt))
Jdf     = m-1;
Jpvalue = 1-chi2.cdf(J_stat, Jdf)
print(J_stat)
print(Jpvalue)